<a href="https://colab.research.google.com/github/Flonsbob/Floris-van-Eisden---Exam---Applied-Statistics-and-Modeling/blob/main/Exam_assignment_Final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imporing all that is neccesary at the very beginning
import pandas as pd

In [2]:
#upload the data
from google.colab import files
files.upload()

Saving adrenal_tumour.csv to adrenal_tumour.csv


{'adrenal_tumour.csv': b'0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,status\n228.53,1554.8,237.34,51.33,72.0,56.17,984.06,93.83,149.27,261.54,166.44,63.8,220.78,1547.7,46.26,368.91,251.85,124.46,256.95,91.88,6.38,734.57,384.9,1084.0,264.41,5.92,360.39,2.27,153.04,61.81,173.95,16.68,CTL\n272.74,1506.8,581.43,113.7,440.65,60.05,1548.2,118.75,113.8,1025.5,1962.7,74.24,382.88,2781.2,43.66,778.75,879.3,169.63,223.12,93.53,24.12,957.95,448.07,2228.3,390.51,7.23,342.35,13.19,141.35,50.09,171.9,113.09,CTL\n264.82,1769.4,98.33,79.73,234.27,43.05,1309.4,155.1,65.93,725.83,268.07,124.63,174.45,4109.9,43.7,346.14,506.0,72.75,293.58,175.82,18.33,805.95,909.2,893.96,165.23,6.78,244.43,6.54,123.03,23.66,194.11,29.12,CTL\n103.07,609.4,169.51,162.96,297.85,2.51,501.28,38.3,28.95,358.07,89.14,40.21,89.41,544.52,0.0,260.15,188.49,88.37,336.91,32.25,60.3,1366.6,177.81,525.5,70.3,1.94,190.84,2.28,26.78,16.63,38.72,15.26,CTL\n100.04,1062.5,100.79,39.67,31.74,32.73,9

In [3]:
#create dataframe (df)
df = pd.read_csv('adrenal_tumour.csv')

In [6]:
df.keys()
#tumor status is in column 32

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', 'status'],
      dtype='object')

In [7]:
df.status
#values of the status column are a 3-character string object

0      CTL
1      CTL
2      CTL
3      CTL
4      CTL
      ... 
230    ACA
231    ACA
232    ACA
233    ACA
234    ACA
Name: status, Length: 235, dtype: object

In [5]:
df.describe()
#the values are probably not normally destributed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
count,234.000000,235.000000,175.000000,235.000000,235.000000,154.000000,235.000000,235.000000,235.000000,235.000000,235.000000,235.000000,234.000000,235.000000,155.000000,235.000000,235.000000,235.000000,235.000000,235.000000,153.000000,235.000000,235.000000,235.000000,175.000000,232.000000,235.000000,155.000000,233.000000,235.000000,235.000000,158.000000
mean,628.052564,2579.014723,2888.757543,8056.661106,515.795553,36.755487,1465.014894,154.255660,177.271404,1323.981404,750.006340,119.710745,2661.757479,3736.295660,546.884258,641.116681,722.488638,592.831617,1082.087872,1614.241894,20.396405,1506.188894,1303.236255,2059.321149,2183.940571,154.736272,508.546596,15.537452,189.324893,233.471319,295.516638,69.621108
std,979.739142,4089.306137,9261.613188,61505.601335,2310.264568,43.241975,1020.853283,193.441592,189.927283,4439.317366,2083.221437,92.303926,21570.177580,3198.998091,1750.267240,761.345484,612.087990,1691.634100,3348.193722,8176.773252,38.907800,2951.017446,1107.287149,3546.092878,10782.481688,1136.212276,688.568093,43.065660,316.456264,597.850927,369.727331,154.716312
min,0.000000,249.080000,25.200000,0.000000,7.450000,0.000000,139.540000,14.560000,15.220000,40.610000,21.380000,15.870000,22.760000,456.820000,0.000000,34.930000,25.520000,12.100000,51.010000,4.870000,0.530000,32.710000,11.680000,77.730000,14.930000,0.000000,69.650000,0.000000,0.000000,7.790000,0.000000,2.520000
25%,211.770000,1059.100000,148.470000,45.035000,65.965000,14.502500,792.685000,64.030000,78.830000,269.070000,114.790000,58.980000,95.002500,1888.800000,82.260000,246.045000,352.555000,135.160000,341.745000,51.000000,4.245000,459.045000,544.835000,655.395000,119.385000,8.145000,206.700000,2.350000,70.000000,50.685000,124.830000,12.140000
50%,390.930000,1610.300000,284.160000,110.000000,158.000000,24.375000,1181.000000,109.040000,131.590000,498.000000,191.000000,89.270000,164.340000,2981.100000,145.180000,412.000000,570.540000,266.890000,556.090000,98.650000,11.450000,925.000000,1010.000000,1157.000000,271.850000,15.420000,317.550000,4.460000,118.130000,85.070000,206.990000,21.410000
75%,637.075000,2357.900000,691.365000,459.385000,344.265000,40.362500,1797.250000,177.780000,199.160000,832.855000,413.270000,155.180000,350.962500,4406.850000,255.245000,752.805000,877.570000,462.000000,887.465000,220.400000,22.520000,1547.000000,1697.200000,2062.550000,568.025000,34.157500,515.775000,10.970000,193.560000,137.800000,353.470000,49.030000
max,7917.200000,33928.000000,77408.000000,812810.000000,26957.000000,276.370000,6099.200000,2035.700000,1388.200000,52059.000000,19615.000000,789.740000,322640.000000,24381.000000,15488.000000,5830.900000,4090.500000,18636.000000,35164.000000,87564.000000,427.680000,34357.000000,7510.700000,37623.000000,122360.000000,15540.000000,5198.700000,373.960000,3407.100000,6049.100000,3336.500000,1446.600000


In [4]:
df.info()
#imputation is necessary, perhaps even deleting some rows 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 33 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       234 non-null    float64
 1   1       235 non-null    float64
 2   2       175 non-null    float64
 3   3       235 non-null    float64
 4   4       235 non-null    float64
 5   5       154 non-null    float64
 6   6       235 non-null    float64
 7   7       235 non-null    float64
 8   8       235 non-null    float64
 9   9       235 non-null    float64
 10  10      235 non-null    float64
 11  11      235 non-null    float64
 12  12      234 non-null    float64
 13  13      235 non-null    float64
 14  14      155 non-null    float64
 15  15      235 non-null    float64
 16  16      235 non-null    float64
 17  17      235 non-null    float64
 18  18      235 non-null    float64
 19  19      235 non-null    float64
 20  20      153 non-null    float64
 21  21      235 non-null    float64
 22  22

In [ ]:
kyk